In [1]:
import os, json, joblib, numpy as np, pandas as pd
from pathlib import Path
import warnings 
warnings.filterwarnings("ignore")

from scipy.spatial.transform import Rotation as R

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

from tensorflow.keras.utils import Sequence, to_categorical, pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (
    Input, Conv1D, BatchNormalization, LayerNormalization, Activation, add, MaxPooling1D, Dropout,
    Bidirectional, LSTM, GlobalAveragePooling1D, Dense, Multiply, Reshape,
    Lambda, Concatenate, GRU, GaussianNoise
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
import tensorflow as tf
import polars as pl

import matplotlib.pyplot as plt

2025-09-01 11:13:18.426740: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-01 11:13:18.822643: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756714398.960767    5218 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756714399.003426    5218 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-01 11:13:19.379912: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
print(tf.config.list_physical_devices("GPU"))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
#print(tf.sysconfig.get_build_info())

In [4]:
print("GPU count:", len(tf.config.list_physical_devices('GPU')))

GPU count: 1


In [5]:
state = 28
import random
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.experimental.numpy.random.seed(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
seed_everything(seed=state)

In [6]:
# (Competition metric will only be imported when TRAINing)
TRAIN = True
DEBUG_GATE = False

RAW_DIR = Path("")
PRETRAINED_DIR = Path("new_model_10_fold")
EXPORT_DIR = Path("hier_ls28")
BATCH_SIZE = 64
PAD_PERCENTILE = 95
LR_INIT = 5e-4
WD = 3e-3
MIXUP_ALPHA = 0.4
EPOCHS = 160
PATIENCE = 40
N_SPLITS = 10
MASKING_PROB = 0.25
GATE_LOSS_WEIGHT = 0.20 # 0.20

print("▶ imports ready · tensorflow", tf.__version__)

▶ imports ready · tensorflow 2.18.0


In [7]:
def remove_gravity_from_acc(acc_data, rot_data):
    acc_values = acc_data[['acc_x', 'acc_y', 'acc_z']].values
    quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    linear_accel = np.zeros_like(acc_values)
    gravity_world = np.array([0, 0, 9.81])
    for i in range(len(acc_values)):
        if np.all(np.isnan(quat_values[i])) or np.all(np.isclose(quat_values[i], 0)):
            linear_accel[i, :] = acc_values[i, :]
            continue
        try:
            rotation = R.from_quat(quat_values[i])
            gravity_sensor_frame = rotation.apply(gravity_world, inverse=True)
            linear_accel[i, :] = acc_values[i, :] - gravity_sensor_frame
        except ValueError:
             linear_accel[i, :] = acc_values[i, :]
    return linear_accel

def calculate_angular_velocity_from_quat(rot_data, time_delta=1/200):
    quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    angular_vel = np.zeros((len(quat_values), 3))
    for i in range(len(quat_values) - 1):
        q_t, q_t_plus_dt = quat_values[i], quat_values[i+1]
        if np.all(np.isnan(q_t)) or np.all(np.isnan(q_t_plus_dt)): continue
        try:
            rot_t = R.from_quat(q_t)
            rot_t_plus_dt = R.from_quat(q_t_plus_dt)
            delta_rot = rot_t.inv() * rot_t_plus_dt
            angular_vel[i, :] = delta_rot.as_rotvec() / time_delta
        except ValueError: pass
    return angular_vel

def calculate_angular_distance(rot_data):
    quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    angular_dist = np.zeros(len(quat_values))
    for i in range(len(quat_values) - 1):
        q1, q2 = quat_values[i], quat_values[i+1]
        if np.all(np.isnan(q1)) or np.all(np.isnan(q2)): continue
        try:
            r1, r2 = R.from_quat(q1), R.from_quat(q2)
            relative_rotation = r1.inv() * r2
            angular_dist[i] = np.linalg.norm(relative_rotation.as_rotvec())
        except ValueError: pass
    return angular_dist

In [8]:
# Tensor Manipulations
def time_sum(x): return K.sum(x, axis=1)
def squeeze_last_axis(x): return tf.squeeze(x, axis=-1)
def expand_last_axis(x): return tf.expand_dims(x, axis=-1)

def se_block(x, reduction=8):
    ch = x.shape[-1]
    se = GlobalAveragePooling1D()(x)
    se = Dense(ch // reduction, activation='relu')(se)
    se = Dense(ch, activation='sigmoid')(se)
    se = Reshape((1, ch))(se)
    return Multiply()([x, se])

In [9]:
class GatedMixupGenerator(Sequence):
    def __init__(self, X, y, batch_size, imu_dim, class_weight=None, alpha=0.2, masking_prob=0.0, shuffle=True):
        self.X, self.y = X, y
        self.batch = batch_size
        self.imu_dim = imu_dim
        self.class_weight = class_weight
        self.alpha = alpha
        self.masking_prob = masking_prob
        self.indices = np.arange(len(X))
        self.shuffle=shuffle
        
    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch))

    def __getitem__(self, i):
        idx = self.indices[i*self.batch:(i+1)*self.batch]
        Xb, yb = self.X[idx].copy(), self.y[idx].copy()
        
        
        sample_weights = np.ones(len(Xb), dtype='float32')
        if self.class_weight:
            y_integers = yb.argmax(axis=1)
            sample_weights = np.array([self.class_weight[i] for i in y_integers])
        
        gate_target = np.ones(len(Xb), dtype='float32')
        if self.masking_prob > 0:
            for i in range(len(Xb)):
                if np.random.rand() < self.masking_prob:
                    Xb[i, :, self.imu_dim:] = 0
                    gate_target[i] = 0.0

        if self.alpha > 0:
            lam = np.random.beta(self.alpha, self.alpha)
            perm = np.random.permutation(len(Xb))
            X_mix = lam * Xb + (1 - lam) * Xb[perm]
            y_mix = lam * yb + (1 - lam) * yb[perm]
            gate_target_mix = lam * gate_target + (1 - lam) * gate_target[perm]
            sample_weights_mix = lam * sample_weights + (1 - lam) * sample_weights[perm]
            return X_mix, {'main_output': y_mix, 'tof_gate': gate_target_mix}, sample_weights_mix

        return Xb, {'main_output': yb, 'tof_gate': gate_target}, sample_weights

    def on_epoch_end(self):
        #if self.shuffle:
        np.random.shuffle(self.indices)

In [10]:
# HIERARCHICAL 0.817LB
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as K

# --- Residual CNN Blok ---
def residual_se_cnn_block(x, filters, kernel_size, drop, wd):
    y = Conv1D(filters, kernel_size, padding='same', use_bias=False, kernel_regularizer=l2(wd))(x)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = SpatialDropout1D(drop)(y)
    y = Conv1D(filters, kernel_size, padding='same', use_bias=False, kernel_regularizer=l2(wd))(y)
    y = BatchNormalization()(y)
    if x.shape[-1] != filters:
        x = Conv1D(filters, 1, padding='same', use_bias=False, kernel_regularizer=l2(wd))(x)
    return Activation('relu')(Add()([x, y]))

# --- Transformer Encoder Bloğu ---
class TransformerEncoderBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.3, wd=5e-4, **kwargs):
        super().__init__(**kwargs)
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim // num_heads)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu", kernel_regularizer=l2(wd)),
            Dense(embed_dim, kernel_regularizer=l2(wd)),
        ])
        self.norm1 = BatchNormalization()
        self.norm2 = BatchNormalization()
        self.drop1 = Dropout(rate)
        self.drop2 = Dropout(rate)

    def call(self, inputs):
        attn_out = self.att(inputs, inputs)
        x = self.norm1(inputs + self.drop1(attn_out))
        ffn_out = self.ffn(x)
        return self.norm2(x + self.drop2(ffn_out))

# --- Positional Embedding ---
class PositionalEmbedding(Layer):
    def __init__(self, sequence_length, output_dim):
        super().__init__()
        self.position_embeddings = Embedding(input_dim=sequence_length, output_dim=output_dim)

    def call(self, inputs):
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        return inputs + self.position_embeddings(positions)

# --- Model ---
def build_hierarchical_transformer_model(pad_len, imu_dim, tof_dim, n_classes, wd=5e-4):
    inp = Input(shape=(pad_len, imu_dim + tof_dim), name='main_input')
    imu_input = Lambda(lambda t: t[:, :, :imu_dim])(inp)
    tof_input = Lambda(lambda t: t[:, :, imu_dim:])(inp)

    # --- IMU ---
    imu = residual_se_cnn_block(imu_input, 64, 3, drop=0.3, wd=wd)
    imu = residual_se_cnn_block(imu, 128, 5, drop=0.3, wd=wd)
    imu = MaxPooling1D(2)(imu)
    imu = MaxPooling1D(2)(imu)
    imu = PositionalEmbedding(imu.shape[1], imu.shape[2])(imu)
    imu = TransformerEncoderBlock(embed_dim=imu.shape[-1], num_heads=4, ff_dim=256, rate=0.3, wd=wd)(imu)

    # --- ToF ---
    tof = Conv1D(64, 3, padding='same', use_bias=False, kernel_regularizer=l2(wd))(tof_input)
    tof = BatchNormalization()(tof)
    tof = Activation('relu')(tof)
    tof = MaxPooling1D(2)(tof)
    tof = SpatialDropout1D(0.3)(tof)
    tof = Conv1D(128, 3, padding='same', use_bias=False, kernel_regularizer=l2(wd))(tof)
    tof = BatchNormalization()(tof)
    tof = Activation('relu')(tof)
    tof = MaxPooling1D(2)(tof)
    tof = SpatialDropout1D(0.4)(tof)
    tof = PositionalEmbedding(tof.shape[1], tof.shape[2])(tof)
    tof = TransformerEncoderBlock(embed_dim=tof.shape[-1], num_heads=4, ff_dim=256, rate=0.3, wd=wd)(tof)

    # --- Cross Attention ---
    imu_to_tof = MultiHeadAttention(num_heads=4, key_dim=tof.shape[-1] // 4)(tof, imu, imu)
    imu_to_tof = Dropout(0.2)(imu_to_tof)
    tof_merged = BatchNormalization()(Add()([tof, imu_to_tof]))

    tof_to_imu = MultiHeadAttention(num_heads=4, key_dim=imu.shape[-1] // 4)(imu, tof, tof)
    tof_to_imu = Dropout(0.2)(tof_to_imu)
    imu_merged = BatchNormalization()(Add()([imu, tof_to_imu]))

    # --- Fuse ---
    merged = Concatenate()([imu_merged, tof_merged])
    gate_input = GlobalAveragePooling1D()(merged)
    gate_input = Dense(32, activation='relu', kernel_regularizer=l2(wd))(gate_input)
    gate = Dense(1, activation='sigmoid', name='tof_gate', kernel_regularizer=l2(wd))(gate_input)
    gate_expanded = RepeatVector(merged.shape[1])(gate)
    gated = Multiply()([merged, gate_expanded])

    # --- Global Context Transformer ---
    context = TransformerEncoderBlock(embed_dim=gated.shape[-1], num_heads=8, ff_dim=256, rate=0.4, wd=wd)(gated)
    x = GlobalAveragePooling1D()(context)

    for units, drop in [(256, 0.5), (128, 0.4)]:
        x = Dense(units, use_bias=False, kernel_regularizer=l2(wd))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(drop)(x)

    out = Dense(n_classes, activation='softmax', name='main_output', kernel_regularizer=l2(wd))(x)

    return Model(inputs=inp, outputs=[out, gate])


In [11]:
from scipy.ndimage import sobel

# ToF için spatial gradyan (sobel) temelli özellikler
def calculate_spatial_tof_features(seq_df, sensor_id):
    # 1D 64-pikseli 8x8'e reshape edip sobel gradyanı alacağız
    pixel_cols = [f"tof_{sensor_id}_v{p}" for p in range(64)]
    tof_data = seq_df[pixel_cols].replace(-1, np.nan).ffill().bfill().fillna(0).values
    
    # Frame sayısı x 64 → (N x 8 x 8)
    N = len(seq_df)
    reshaped = tof_data.reshape(N, 8, 8)
    
    # Spatial gradyanları hesapla (sobel x ve y)
    sobel_x = sobel(reshaped, axis=1)
    sobel_y = sobel(reshaped, axis=2)
    grad_mag = np.sqrt(sobel_x ** 2 + sobel_y ** 2)

    # Özet istatistikleri hesapla
    grad_mean = grad_mag.mean(axis=(1, 2))
    grad_std  = grad_mag.std(axis=(1, 2))
    grad_max  = grad_mag.max(axis=(1, 2))
    
    return pd.DataFrame({
        f'tof_{sensor_id}_grad_mean': grad_mean,
        f'tof_{sensor_id}_grad_std': grad_std,
        f'tof_{sensor_id}_grad_max': grad_max
    }, index=seq_df.index)

In [12]:
if TRAIN: 
    print("▶ TRAIN MODE – loading dataset ...")
    df = pd.read_csv(RAW_DIR / "train.csv")
    
    train_dem_df = pd.read_csv(RAW_DIR / "train_demographics.csv")

    le = LabelEncoder()
    df['gesture_int'] = le.fit_transform(df['gesture'])
    np.save(EXPORT_DIR / "gesture_classes.npy", le.classes_)

    acc_y_neg_subjects = (
        df.groupby('subject')['acc_y']
        .mean()
        .loc[lambda x: x < 0]
        .index
        .tolist()
    )

    print("acc_y ortalaması negatif olan subject'ler:", acc_y_neg_subjects)
    df = df[~df['subject'].isin(acc_y_neg_subjects)].reset_index(drop=True)
    
    print("  Removing gravity and calculating linear acceleration features...")
    linear_accel_list = [pd.DataFrame(remove_gravity_from_acc(group[['acc_x', 'acc_y', 'acc_z']], group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]), columns=['linear_acc_x', 'linear_acc_y', 'linear_acc_z'], index=group.index) for _, group in df.groupby('sequence_id')]
    df = pd.concat([df, pd.concat(linear_accel_list)], axis=1)
    
    df['linear_acc_mag'] = np.sqrt(df['linear_acc_x']**2 + df['linear_acc_y']**2 + df['linear_acc_z']**2)
    df['linear_acc_mag_jerk'] = df.groupby('sequence_id')['linear_acc_mag'].diff().fillna(0)
    
    print("  Calculating angular velocity and distance from quaternions...")
    angular_vel_list = [pd.DataFrame(calculate_angular_velocity_from_quat(group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]), columns=['angular_vel_x', 'angular_vel_y', 'angular_vel_z'], index=group.index) for _, group in df.groupby('sequence_id')]
    df = pd.concat([df, pd.concat(angular_vel_list)], axis=1)
    angular_dist_list = [pd.DataFrame(calculate_angular_distance(group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]), columns=['angular_distance'], index=group.index) for _, group in df.groupby('sequence_id')]
    df = pd.concat([df, pd.concat(angular_dist_list)], axis=1)

    for col in ['acc_x', 'acc_y', 'acc_z',  'linear_acc_x', 'linear_acc_y', 'linear_acc_z', 'angular_vel_x', 'angular_vel_y', 'angular_vel_z']:
        if col in df.columns:
            df[f'{col}_diff'] = df.groupby('sequence_id')[col].diff().fillna(0)
            df[f'{col}_abs_diff'] = np.abs(df.groupby('sequence_id')[col].diff()).fillna(0)

    imu_cols_base = ['acc_x', 'acc_y', 'acc_z'] + [c for c in df.columns if c.startswith('rot_')]
    imu_engineered = [
    'linear_acc_mag', 'linear_acc_mag_jerk',
    'angular_vel_x', 'angular_vel_y', 'angular_vel_z', 'angular_distance'
    ]
    for col in ['acc_x', 'acc_y', 'acc_z', 'linear_acc_x', 'linear_acc_y', 'linear_acc_z', 'angular_vel_x', 'angular_vel_y', 'angular_vel_z']:
        if col in df.columns:
            imu_engineered.append(f'{col}_diff')
            imu_engineered.append(f'{col}_abs_diff')
    imu_cols = list(dict.fromkeys(imu_cols_base + imu_engineered))
    
    thm_cols_original = [c for c in df.columns if c.startswith('thm_')]
    
    tof_aggregated_cols_template = []
    for i in range(1, 6): tof_aggregated_cols_template.extend([f'tof_{i}_mean', f'tof_{i}_std', f'tof_{i}_min', f'tof_{i}_max'])

    for i in range(1, 6):
        tof_aggregated_cols_template.extend([
            f'tof_{i}_grad_mean', f'tof_{i}_grad_std', f'tof_{i}_grad_max'
        ])
    
    final_feature_cols = imu_cols + thm_cols_original + tof_aggregated_cols_template

    imu_dim_final = len(imu_cols)
    tof_thm_aggregated_dim_final = len(thm_cols_original) + len(tof_aggregated_cols_template)
    
    print(f"  IMU (phys-based + enhanced) {imu_dim_final} | THM + Aggregated TOF {tof_thm_aggregated_dim_final} | total {len(final_feature_cols)} features")
    np.save(EXPORT_DIR / "feature_cols.npy", np.array(final_feature_cols))

    print("  Building sequences...")
    seq_gp = df.groupby('sequence_id')
    X_list_unscaled, y_list_int, groups_list, lens = [], [], [], []
    for seq_id, seq_df in seq_gp:
        seq_df_copy = seq_df.copy()
        for i in range(1, 6):
            pixel_cols = [f"tof_{i}_v{p}" for p in range(64)]; tof_data = seq_df_copy[pixel_cols].replace(-1, np.nan)
            seq_df_copy[f'tof_{i}_mean'], seq_df_copy[f'tof_{i}_std'], seq_df_copy[f'tof_{i}_min'], seq_df_copy[f'tof_{i}_max'] = tof_data.mean(axis=1), tof_data.std(axis=1), tof_data.min(axis=1), tof_data.max(axis=1)
            
            spatial_feats = calculate_spatial_tof_features(seq_df_copy, i)
            seq_df_copy = pd.concat([seq_df_copy, spatial_feats], axis=1)
        
        X_list_unscaled.append(seq_df_copy[final_feature_cols].ffill().bfill().fillna(0).values.astype('float32'))
        y_list_int.append(seq_df_copy['gesture_int'].iloc[0])
        groups_list.append(seq_df_copy['subject'].iloc[0])
        lens.append(len(seq_df_copy))

    print("  Fitting StandardScaler...")
    all_steps_concatenated = np.concatenate(X_list_unscaled, axis=0)
    scaler = StandardScaler().fit(all_steps_concatenated)
    joblib.dump(scaler, EXPORT_DIR / "scaler.pkl")
    
    print("  Scaling and padding sequences...")
    X_scaled_list = [scaler.transform(x_seq) for x_seq in X_list_unscaled]
    pad_len = int(np.percentile(lens, PAD_PERCENTILE)); np.save(EXPORT_DIR / "sequence_maxlen.npy", pad_len)
    X = pad_sequences(X_scaled_list, maxlen=pad_len, padding='post', truncating='post', dtype='float32')

    subject_acc_x_mean_global = df.groupby('subject')['acc_x'].mean()
    subject_is_acc_x_mean_negative = (subject_acc_x_mean_global < 0).astype(str)
    
    y_stratify = np.array([f"{gesture_label}_{subject_is_acc_x_mean_negative.loc[sub_id]}"
                           for gesture_label, sub_id in zip(y_list_int, groups_list)])
    
    groups, y = np.array(groups_list), to_categorical(y_list_int, num_classes=len(le.classes_))
    print("  Starting training with Stratified Group K-Fold CV...")
    sgkf = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True, random_state=state) # state_num yerine state kullanıldı
    oof_preds = np.zeros_like(y, dtype='float32')
    
    for fold, (train_idx, val_idx) in enumerate(sgkf.split(X, y_stratify, groups)):
        print(f"\n===== FOLD {fold+1}/{N_SPLITS} =====")
        X_tr, X_val, y_tr, y_val = X[train_idx], X[val_idx], y[train_idx], y[val_idx]# y_val düzeltildi
        
        # --- DEĞİŞİKLİK BAŞLANGICI ---
        # Modelinizi burada çağırın
        model = build_hierarchical_transformer_model(
            pad_len, imu_dim_final, tof_thm_aggregated_dim_final, len(le.classes_), wd=WD
        )

        # Custom katmanları compile ve save/load için kaydet.
        # Bu custom_objects, model.save() ve tf.keras.models.load_model() için gereklidir.
        #custom_objects_for_model = {
         #   'TransformerEncoderBlock': TransformerEncoderBlock,
        #    'PositionalEmbedding': PositionalEmbedding,
            # Eğer residual_se_cnn_block ve attention_layer custom Layer ise, onları da ekleyin.
            # Şu anki tanımlarınız Layer sınıfından kalıtım almadığı için gerekmez,
            # ancak Layer olarak yeniden yazarsanız eklersiniz.
       # }
        # --- DEĞİŞİKLİK SONU ---

        lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_main_output_accuracy',
            mode='max',
            factor=0.5,
            patience=8,
            cooldown=2,
            min_lr=3e-6,
            verbose=1
        )
        
     
        
        model.compile(optimizer=Adam(LR_INIT),
                      loss={'main_output': tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
                             'tof_gate': tf.keras.losses.BinaryCrossentropy()
                             },
                      loss_weights={'main_output': 1.0,
                                     'tof_gate': GATE_LOSS_WEIGHT,
                                     },
                      metrics={'main_output': 'accuracy'})
        
        class_weight_dict = dict(enumerate(compute_class_weight('balanced', classes=np.arange(len(le.classes_)), y=y_tr.argmax(1))))
        
        train_gen = GatedMixupGenerator(X_tr, y_tr, batch_size=BATCH_SIZE, imu_dim=imu_dim_final, class_weight=class_weight_dict, alpha=MIXUP_ALPHA, masking_prob=MASKING_PROB, shuffle=True)
        val_gen = GatedMixupGenerator(X_val, y_val, batch_size=BATCH_SIZE, imu_dim=imu_dim_final,class_weight=None, alpha=0.2, shuffle=False) # İmu_dim burada da doğru olmalı

       

        cb = [
            EarlyStopping(patience=PATIENCE, restore_best_weights=True, verbose=1, monitor='val_main_output_accuracy', mode='max'),
            lr_scheduler
        ]
        
        model.fit(train_gen, epochs=EPOCHS, validation_data=val_gen, callbacks=cb, verbose=1)
        
        # --- DEĞİŞİKLİK BAŞLANGICI ---
        # Modeli custom_objects ile kaydedin
        model_save_path = EXPORT_DIR / f"gesture_model_fold_{fold}" # .h5 uzantısı olmadan bir dizin adı
        tf.saved_model.save(model, str(model_save_path)) # SavedModel formatında kaydet
        print(f"Model kaydedildi: {model_save_path}")
        # --- DEĞİŞİKLİK SONU ---

        preds_val, _ = model.predict(X_val)
        oof_preds[val_idx] = preds_val

    print("\n✔ Training done.")
    
    from metric import CompetitionMetric # Import path needs to be correct
    true_oof_int = y.argmax(1)
    pred_oof_int = oof_preds.argmax(1)
        
    h_f1_oof = CompetitionMetric().calculate_hierarchical_f1(
        pd.DataFrame({'gesture': le.classes_[true_oof_int]}),
        pd.DataFrame({'gesture': le.classes_[pred_oof_int]}))
    print(f"Overall OOF H‑F1 Score = {h_f1_oof:.4f}")
 

▶ TRAIN MODE – loading dataset ...
acc_y ortalaması negatif olan subject'ler: ['SUBJ_019262', 'SUBJ_045235']
  Removing gravity and calculating linear acceleration features...
  Calculating angular velocity and distance from quaternions...
  IMU (phys-based + enhanced) 31 | THM + Aggregated TOF 40 | total 71 features
  Building sequences...
  Fitting StandardScaler...
  Scaling and padding sequences...
  Starting training with Stratified Group K-Fold CV...

===== FOLD 1/10 =====


I0000 00:00:1756714550.767424    5218 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21770 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/160


I0000 00:00:1756714559.256584    5872 cuda_dnn.cc:529] Loaded cuDNN version 90300


112/112 ━━━━━━━━━━━━━━━━━━━━ 23s 128ms/step - loss: 10.0450 - main_output_accuracy: 0.0904 - main_output_loss: 3.2480 - tof_gate_loss: 0.6515 - val_loss: 8.6277 - val_main_output_accuracy: 0.2120 - val_main_output_loss: 2.6453 - val_tof_gate_loss: 0.3153 - learning_rate: 5.0000e-04
Epoch 2/160
112/112 ━━━━━━━━━━━━━━━━━━━━ 12s 110ms/step - loss: 8.5331 - main_output_accuracy: 0.1422 - main_output_loss: 2.8321 - tof_gate_loss: 0.3472 - val_loss: 7.2601 - val_main_output_accuracy: 0.2757 - val_main_output_loss: 2.4027 - val_tof_gate_loss: 0.1099 - learning_rate: 5.0000e-04
Epoch 3/160
112/112 ━━━━━━━━━━━━━━━━━━━━ 14s 122ms/step - loss: 7.2728 - main_output_accuracy: 0.1874 - main_output_loss: 2.6243 - tof_gate_loss: 0.3400 - val_loss: 6.1374 - val_main_output_accuracy: 0.3493 - val_main_output_loss: 2.2553 - val_tof_gate_loss: 0.0847 - learning_rate: 5.0000e-04
Epoch 4/160
112/112 ━━━━━━━━━━━━━━━━━━━━ 14s 123ms/step - loss: 6.1216 - main_output_accuracy: 0.2520 - main_output_loss: 2.4057 

...

26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step

===== FOLD 10/10 =====
Epoch 1/160
114/114 ━━━━━━━━━━━━━━━━━━━━ 23s 136ms/step - loss: 9.9993 - main_output_accuracy: 0.0845 - main_output_loss: 3.1981 - tof_gate_loss: 0.7271 - val_loss: 8.5597 - val_main_output_accuracy: 0.2102 - val_main_output_loss: 2.6098 - val_tof_gate_loss: 0.3285 - learning_rate: 5.0000e-04
Epoch 2/160
114/114 ━━━━━━━━━━━━━━━━━━━━ 12s 102ms/step - loss: 8.4769 - main_output_accuracy: 0.1632 - main_output_loss: 2.7918 - tof_gate_loss: 0.3900 - val_loss: 7.2852 - val_main_output_accuracy: 0.2492 - val_main_output_loss: 2.4772 - val_tof_gate_loss: 0.1191 - learning_rate: 5.0000e-04
Epoch 3/160
114/114 ━━━━━━━━━━━━━━━━━━━━ 14s 127ms/step - loss: 7.1436 - main_output_accuracy: 0.2095 - main_output_loss: 2.5525 - tof_gate_loss: 0.3106 - val_loss: 6.1483 - val_main_output_accuracy: 0.3213 - val_main_output_loss: 2.3238 - val_tof_gate_loss: 0.0923 - learning_rate: 5.0000e-04
Epoch 4/160
114/114 ━━━━━━━━━━━━━━━━━━━━ 12s 105ms/ste

INFO:tensorflow:Assets written to: hier_ls28/gesture_model_fold_9/assets


Model kaydedildi: hier_ls28/gesture_model_fold_9


2025-09-01 16:31:24.328293: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step

✔ Training done.
Overall OOF H‑F1 Score = 0.8356


In [13]:
def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    df_seq = sequence.to_pandas()
    seq_df_copy = df_seq.copy() 


    linear_accel = remove_gravity_from_acc(df_seq, df_seq)
    df_seq['linear_acc_x'], df_seq['linear_acc_y'], df_seq['linear_acc_z'] = linear_accel[:, 0], linear_accel[:, 1], linear_accel[:, 2]
    df_seq['linear_acc_mag'] = np.sqrt(df_seq['linear_acc_x']**2 + df_seq['linear_acc_y']**2 + df_seq['linear_acc_z']**2)
    df_seq['linear_acc_mag_jerk'] = df_seq['linear_acc_mag'].diff().fillna(0)
    angular_vel = calculate_angular_velocity_from_quat(df_seq)
    df_seq['angular_vel_x'], df_seq['angular_vel_y'], df_seq['angular_vel_z'] = angular_vel[:, 0], angular_vel[:, 1], angular_vel[:, 2]
    df_seq['angular_distance'] = calculate_angular_distance(df_seq)

    for col in ['acc_x', 'acc_y', 'acc_z', 'linear_acc_x', 'linear_acc_y', 'linear_acc_z', 'angular_vel_x', 'angular_vel_y', 'angular_vel_z']:
        if col in df_seq.columns:
            df_seq[f'{col}_diff'] = df_seq.groupby('sequence_id')[col].diff().fillna(0)
            df_seq[f'{col}_abs_diff'] = np.abs(df_seq.groupby('sequence_id')[col].diff()).fillna(0) # Mutlak fark

    for i in range(1, 6):
        pixel_cols = [f"tof_{i}_v{p}" for p in range(64)]; tof_data = df_seq[pixel_cols].replace(-1, np.nan)
        df_seq[f'tof_{i}_mean'], df_seq[f'tof_{i}_std'], df_seq[f'tof_{i}_min'], df_seq[f'tof_{i}_max'] = tof_data.mean(axis=1), tof_data.std(axis=1), tof_data.min(axis=1), tof_data.max(axis=1)
        spatial_feats = calculate_spatial_tof_features(seq_df_copy, i)
        df_seq = pd.concat([df_seq, spatial_feats], axis=1)
        
        
    mat_unscaled = df_seq[final_feature_cols].ffill().bfill().fillna(0).values.astype('float32')
    mat_scaled = scaler.transform(mat_unscaled)
    pad_input = pad_sequences([mat_scaled], maxlen=pad_len, padding='post', truncating='post', dtype='float32')

   
    all_preds = [model.predict(pad_input, verbose=0)[0] for model in models] # 主出力のみ取得
    avg_pred = np.mean(all_preds, axis=0)
    print(str(gesture_classes[avg_pred.argmax()]))
    return str(gesture_classes[avg_pred.argmax()])


In [14]:
if not TRAIN:
    import kaggle_evaluation.cmi_inference_server
    inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        inference_server.serve()
    else:
        inference_server.run_local_gateway(
            data_paths=(
                '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
                '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
            )
        )